# Check meta-model training as binary predictor

In [1]:
# https://ipython.org/ipython-doc/3/config/extensions/autoreload.html
%load_ext autoreload
%autoreload 2

from pathlib import Path
import sys
sys.path.insert(0, Path(".").absolute().parent.as_posix())

from PIL import Image

import pandas as pd

import numpy as np
from common.dataset import FilesFromCsvDataset, TransformedDataset
from common.meta import get_metafeatures, get_imsize_and_targets

In [2]:
import matplotlib.pylab as plt
%matplotlib inline
import seaborn as sns

sns.set_style('darkgrid')
sns.set(rc={'figure.figsize':(12, 10)})

In [3]:
meta_features_path = Path("../output")
meta_features_list = [
    meta_features_path / "val_probas_inceptionresnetv2_350_resized_crop" / "20180428_1622" / "probas.csv",
    meta_features_path / "val_probas_inceptionv4_350_resized_crop" / "20180428_1633" / "probas.csv",
    meta_features_path / "val_probas_nasnetalarge_350_resized_crop" / "20180428_1654" / "probas.csv",
]

## Prepare data

In [5]:
def create_topk_with_probas_df(df, k):
    df_values = df.values
    topk_values = np.argsort(df_values, axis=1)[:, -k:]
    topk_probas = np.zeros_like(topk_values, dtype=np.float)
    for i, indices in enumerate(topk_values):
        topk_probas[i, :] = df_values[i, indices]
    cols1 = ["top_{}".format(k - i) for i in range(k)]
    cols2 = ["top_{}_proba".format(k - i) for i in range(k)]
    data = np.concatenate([topk_values, topk_probas], axis=1)
    topk_df = pd.DataFrame(data, index=df.index, columns=cols1 + cols2)
    topk_df[cols1] = topk_df[cols1].astype(np.int)
    return topk_df


def get_topk_with_probas_metafeatures(prediction_files, k=5):
    dfs = [pd.read_csv(f, index_col='id') for f in prediction_files]
    dfs = [create_topk_with_probas_df(df, k=k) for df in dfs]
    for i, df in enumerate(dfs):
        df.columns = ["f{}_{}".format(i, c) for c in df.columns]
    meta_features = pd.concat([df for df in dfs], axis=1)
    return meta_features

In [7]:
n_models = len(meta_features_list)
k = 5

In [8]:
meta_features_init = get_topk_with_probas_metafeatures(meta_features_list, k=k)

In [9]:
meta_features_init.head()

,f0_top_5,f0_top_4,f0_top_3,f0_top_2,f0_top_1,f0_top_5_proba,f0_top_4_proba,f0_top_3_proba,f0_top_2_proba,f0_top_1_proba,...,f2_top_5,f2_top_4,f2_top_3,f2_top_2,f2_top_1,f2_top_5_proba,f2_top_4_proba,f2_top_3_proba,f2_top_2_proba,f2_top_1_proba
id,,,,,,,,,,,,,,,,,,,,,
6302,36,83,96,52,47,2.801312e-08,1.670775e-07,5.467300e-07,0.000012,0.999987,...,81,13,83,52,47,0.000011,0.000013,0.000014,0.000063,0.999842
3349,122,71,8,106,78,3.666026e-04,1.514090e-03,4.313464e-03,0.014010,0.979373,...,8,56,71,106,78,0.005425,0.009689,0.010681,0.112638,0.859415
484,2,68,42,21,118,1.504418e-05,1.539339e-05,3.831230e-05,0.000168,0.999716,...,2,20,44,21,118,0.000217,0.000359,0.002980,0.005161,0.988247
2677,48,104,99,111,26,3.810589e-06,1.201251e-05,5.372663e-05,0.001435,0.998493,...,73,81,99,111,26,0.000020,0.000031,0.000061,0.000239,0.999580
1517,91,20,21,119,16,2.052679e-03,2.681254e-03,2.196829e-02,0.104285,0.865519,...,44,20,21,119,16,0.004991,0.013037,0.029268,0.064841,0.879447


In [10]:
dataset = FilesFromCsvDataset("../output/filtered_val_dataset.csv")
dataset = TransformedDataset(dataset,
                             transforms=lambda x: (x, Image.open(x).size),
                             target_transforms=lambda l: l - 1)
df_imsize_targets = get_imsize_and_targets(dataset)

In [11]:
print(df_imsize_targets.shape)
df_imsize_targets.head()

(6291, 3)


,height,target,width
6302,800,47,800
3349,550,78,730
484,800,118,800
2677,480,26,640
1517,800,21,800


In [14]:
y_true = df_imsize_targets['target']
y_topk_probas = meta_features_init.loc[y_true.index, :]
y_true.name = 'y_true'

meta_trainval = pd.concat([meta_features_init, y_true], axis=1)
meta_trainval.tail(10)

,f0_top_5,f0_top_4,f0_top_3,f0_top_2,f0_top_1,f0_top_5_proba,f0_top_4_proba,f0_top_3_proba,f0_top_2_proba,f0_top_1_proba,...,f2_top_4,f2_top_3,f2_top_2,f2_top_1,f2_top_5_proba,f2_top_4_proba,f2_top_3_proba,f2_top_2_proba,f2_top_1_proba,y_true
6391,46,37,20,16,91,2.900470e-05,6.800265e-05,6.920366e-05,1.008632e-03,0.998755,...,37,119,16,91,1.325156e-04,1.343532e-04,2.051220e-04,6.789478e-03,0.992390,91.0
6392,4,86,38,117,107,1.404220e-04,1.883191e-04,2.486526e-04,6.593229e-03,0.992687,...,4,86,117,107,2.460717e-04,2.479940e-04,2.532245e-04,1.266720e-02,0.986045,107.0
6393,97,113,31,104,59,2.889563e-05,4.944770e-05,9.245931e-05,2.962509e-01,0.703197,...,124,24,104,59,4.215291e-03,4.987243e-03,5.541493e-03,2.166809e-01,0.700548,104.0
6394,19,1,49,87,53,1.506875e-03,1.852114e-03,1.445668e-02,1.393662e-01,0.842521,...,1,87,49,53,9.761274e-04,1.189077e-02,1.981492e-02,3.459351e-02,0.931513,49.0
6395,31,24,61,60,121,3.292141e-04,8.314594e-04,8.937456e-04,4.017614e-02,0.956916,...,33,61,60,121,1.451288e-03,1.648911e-03,3.247719e-03,5.519204e-02,0.928640,121.0
6396,42,7,107,40,15,8.513351e-09,8.681659e-09,1.461853e-08,2.904834e-08,1.000000,...,1,107,68,15,1.410164e-08,1.777103e-08,4.600141e-08,2.970329e-07,1.000000,15.0
6397,73,109,126,26,111,3.731116e-03,2.527305e-02,1.554033e-01,2.624698e-01,0.548090,...,81,126,111,26,4.660923e-04,5.928035e-04,2.070051e-03,4.434516e-01,0.552211,111.0
6398,97,49,1,87,53,6.605026e-05,8.199667e-04,1.187497e-01,1.994584e-01,0.680775,...,49,87,1,53,1.424019e-03,4.622611e-03,1.373499e-01,2.587718e-01,0.596021,87.0
6399,23,119,30,84,61,2.323027e-05,3.819728e-05,6.817383e-05,9.479532e-03,0.990335,...,60,30,84,61,2.975727e-04,3.853616e-04,6.003686e-04,1.160517e-01,0.881931,61.0
6400,106,11,78,71,122,2.460118e-03,3.029029e-03,3.654767e-03,2.254799e-02,0.964166,...,71,106,78,122,2.787408e-06,5.724145e-06,4.190853e-05,1.152824e-04,0.999828,122.0


In [16]:
topk_cols = [c for c in meta_trainval.columns.tolist() if c[-5:-1] == "top_"]

In [15]:
meta_trainval.head()

,f0_top_5,f0_top_4,f0_top_3,f0_top_2,f0_top_1,f0_top_5_proba,f0_top_4_proba,f0_top_3_proba,f0_top_2_proba,f0_top_1_proba,...,f2_top_4,f2_top_3,f2_top_2,f2_top_1,f2_top_5_proba,f2_top_4_proba,f2_top_3_proba,f2_top_2_proba,f2_top_1_proba,y_true
1,16,28,100,91,37,6.012118e-08,3.154609e-07,5.833772e-07,9.576799e-07,0.999998,...,28,30,91,37,0.000006,0.000006,0.000010,0.000082,0.999839,37.0
2,25,102,28,14,62,1.674296e-02,5.781974e-02,7.877417e-02,1.107072e-01,0.718856,...,102,14,28,62,0.006253,0.060232,0.112365,0.191582,0.603135,62.0
3,77,104,123,64,32,4.137828e-04,7.174005e-04,8.619499e-04,1.559486e-03,0.995548,...,27,10,64,32,0.000102,0.000121,0.000137,0.000450,0.998686,32.0
4,22,3,62,14,125,5.834689e-04,1.269720e-03,7.635363e-03,1.163034e-02,0.977478,...,22,62,14,125,0.000990,0.001900,0.019804,0.053805,0.919623,125.0
5,93,3,20,21,17,1.826174e-06,1.104301e-05,5.581508e-05,1.505214e-04,0.999775,...,44,93,20,17,0.000003,0.000007,0.000007,0.000060,0.999905,17.0


In [20]:
meta_trainval[topk_cols].apply(lambda row: set(row), axis=1)

In [43]:
data = []
for i in range(1, k + 1):
    d = meta_features.copy()
    d.loc[:, "pos_class_id"] = d["top_{}".format(i)]
    data.append(d)

meta_trainval_ = pd.concat(data, axis=0)

In [44]:
meta_trainval_.loc[100, :]

,top_5,top_4,top_3,top_2,top_1,top_5_proba,top_4_proba,top_3_proba,top_2_proba,top_1_proba,y_true,pos_class_id
100,80,10,32,51,76,0.00182,0.002202,0.009819,0.012623,0.973327,76,76
100,80,10,32,51,76,0.00182,0.002202,0.009819,0.012623,0.973327,76,51
100,80,10,32,51,76,0.00182,0.002202,0.009819,0.012623,0.973327,76,32
100,80,10,32,51,76,0.00182,0.002202,0.009819,0.012623,0.973327,76,10
100,80,10,32,51,76,0.00182,0.002202,0.009819,0.012623,0.973327,76,80


In [45]:
meta_trainval_.loc[:, 'target'] = (meta_trainval_['y_true'] == meta_trainval_['pos_class_id']).astype(np.int)

In [105]:
print(meta_trainval_.shape)
meta_trainval_.loc[3333, :]

(31455, 13)


,top_5,top_4,top_3,top_2,top_1,top_5_proba,top_4_proba,top_3_proba,top_2_proba,top_1_proba,y_true,pos_class_id,target
3333,72,86,76,6,51,0.060209,0.103765,0.119072,0.130597,0.463619,76,51,0
3333,72,86,76,6,51,0.060209,0.103765,0.119072,0.130597,0.463619,76,6,0
3333,72,86,76,6,51,0.060209,0.103765,0.119072,0.130597,0.463619,76,76,1
3333,72,86,76,6,51,0.060209,0.103765,0.119072,0.130597,0.463619,76,86,0
3333,72,86,76,6,51,0.060209,0.103765,0.119072,0.130597,0.463619,76,72,0


In [47]:
train_columns = meta_trainval_.columns.tolist()
train_columns.remove('y_true')
train_columns.remove('target')

In [120]:
len(meta_trainval_) / 5

6291.0

In [123]:
meta_trainval_[mask_top].shape[0] / (len(meta_trainval_) / 5)

0.8772850103322206

In [124]:
mask_top = (meta_trainval_['pos_class_id'] == meta_trainval_['top_1']) & (meta_trainval_['target'] == 1)
print(meta_trainval_[mask_top].shape, meta_trainval_[mask_top].shape[0] / (len(meta_trainval_) / 5))
meta_trainval_[mask_top].head()

(5519, 13) 0.8772850103322206


,top_5,top_4,top_3,top_2,top_1,top_5_proba,top_4_proba,top_3_proba,top_2_proba,top_1_proba,y_true,pos_class_id,target
6302,81,13,83,52,47,0.000004,0.000004,0.000005,0.000025,0.999943,47,47,1
3349,56,71,8,106,78,0.003627,0.004472,0.004776,0.047875,0.938301,78,78,1
484,2,20,44,21,118,0.000077,0.000120,0.000996,0.001776,0.995987,118,118,1
2677,48,81,99,111,26,0.000007,0.000010,0.000038,0.000560,0.999356,26,26,1
5908,85,37,100,69,116,0.000065,0.000096,0.000111,0.014535,0.984703,116,116,1


In [125]:
mask_top = (meta_trainval_['pos_class_id'] == meta_trainval_['top_2']) & (meta_trainval_['target'] == 1)
print(meta_trainval_[mask_top].shape, meta_trainval_[mask_top].shape[0] / (len(meta_trainval_) / 5))
meta_trainval_[mask_top].head()

(480, 13) 0.07629947544110634


,top_5,top_4,top_3,top_2,top_1,top_5_proba,top_4_proba,top_3_proba,top_2_proba,top_1_proba,y_true,pos_class_id,target
916,43,101,31,65,39,2.959798e-02,4.417373e-02,2.398183e-01,0.289503,0.381428,65,65,1
4853,24,45,78,56,106,6.592789e-04,1.644349e-03,5.020495e-03,0.105461,0.885488,56,56,1
3126,39,11,88,29,36,9.423055e-08,1.046994e-07,4.866039e-07,0.000001,0.999998,29,29,1
3175,2,3,25,22,62,3.258823e-02,5.591428e-02,8.045318e-02,0.314523,0.466907,22,22,1
638,119,104,127,84,59,1.811040e-03,2.163361e-03,3.847145e-03,0.128065,0.862018,84,84,1


In [126]:
mask_top = (meta_trainval_['pos_class_id'] == meta_trainval_['top_3']) & (meta_trainval_['target'] == 1)
print(meta_trainval_[mask_top].shape, meta_trainval_[mask_top].shape[0] / (len(meta_trainval_) / 5))
meta_trainval_[mask_top].head()

(140, 13) 0.022254013670322682


,top_5,top_4,top_3,top_2,top_1,top_5_proba,top_4_proba,top_3_proba,top_2_proba,top_1_proba,y_true,pos_class_id,target
1517,44,20,21,119,16,0.002835,0.010127,0.023157,0.059811,0.897837,21,21,1
347,108,117,107,86,38,0.007261,0.043711,0.091113,0.110699,0.746043,107,107,1
4584,127,95,7,18,102,0.004364,0.027536,0.037916,0.052563,0.875009,7,7,1
1142,64,121,120,60,61,0.053782,0.070356,0.095659,0.127539,0.480640,120,120,1
125,95,127,102,18,7,0.022071,0.048307,0.073341,0.241550,0.604248,102,102,1


In [127]:
mask_top = (meta_trainval_['pos_class_id'] == meta_trainval_['top_4']) & (meta_trainval_['target'] == 1)
print(meta_trainval_[mask_top].shape, meta_trainval_[mask_top].shape[0] / (len(meta_trainval_) / 5))
meta_trainval_[mask_top].head()

(59, 13) 0.009378477189635987


,top_5,top_4,top_3,top_2,top_1,top_5_proba,top_4_proba,top_3_proba,top_2_proba,top_1_proba,y_true,pos_class_id,target
4998,12,127,79,18,7,0.007701,0.074280,0.103385,0.185535,0.614360,127,127,1
1637,3,14,28,22,62,0.058805,0.085025,0.156178,0.204352,0.486785,14,14,1
3121,22,90,34,21,44,0.003845,0.008967,0.009490,0.160142,0.806794,90,90,1
4255,97,49,53,87,1,0.002789,0.007316,0.057496,0.252311,0.679545,49,49,1
2080,97,49,1,87,53,0.001630,0.038874,0.047671,0.054677,0.856167,49,49,1


In [128]:
mask_top = (meta_trainval_['pos_class_id'] == meta_trainval_['top_5']) & (meta_trainval_['target'] == 1)
print(meta_trainval_[mask_top].shape, meta_trainval_[mask_top].shape[0] / (len(meta_trainval_) / 5))
meta_trainval_[mask_top].head()

(21, 13) 0.0033381020505484026


,top_5,top_4,top_3,top_2,top_1,top_5_proba,top_4_proba,top_3_proba,top_2_proba,top_1_proba,y_true,pos_class_id,target
5102,116,69,74,114,41,0.072114,0.083198,0.090794,0.114956,0.510313,116,116,1
3871,29,50,88,52,96,0.007500,0.010138,0.038395,0.441316,0.498914,29,29,1
5292,26,111,124,126,81,0.043802,0.044589,0.067075,0.304749,0.524564,26,26,1
895,58,18,102,7,125,0.021013,0.058851,0.079689,0.202766,0.509786,58,58,1
606,88,50,29,96,52,0.005366,0.009776,0.044885,0.124369,0.810559,88,88,1


In [74]:
from sklearn.metrics import accuracy_score

accuracy_score(meta_trainval_['y_true'], meta_trainval_['top_1'])

0.8772850103322206

Limit to top 2

In [264]:
k = 2
meta_trainval.loc[:, "pos_class_id"] = meta_trainval["top_1"]
data = [meta_trainval, ] 
for i in range(2, k + 1):
    d = meta_trainval.copy()
    d.loc[:, "pos_class_id"] = d["top_{}".format(i)]
    data.append(d)

meta_trainval_ = pd.concat(data, axis=0)

meta_trainval_.loc[:, 'target'] = (meta_trainval_['y_true'] == meta_trainval_['pos_class_id']).astype(np.int)

In [265]:
meta_trainval_.head()

,top_5,top_4,top_3,top_2,top_1,top_5_proba,top_4_proba,top_3_proba,top_2_proba,top_1_proba,y_true,pos_class_id,target
6302,81,13,83,52,47,0.000004,0.000004,0.000005,0.000025,0.999943,47,47,1
3349,56,71,8,106,78,0.003627,0.004472,0.004776,0.047875,0.938301,78,78,1
484,2,20,44,21,118,0.000077,0.000120,0.000996,0.001776,0.995987,118,118,1
2677,48,81,99,111,26,0.000007,0.000010,0.000038,0.000560,0.999356,26,26,1
1517,44,20,21,119,16,0.002835,0.010127,0.023157,0.059811,0.897837,21,16,0


In [341]:
meta_trainval_2 = pd.concat([meta_trainval_, pd.get_dummies(meta_trainval_['pos_class_id'], prefix='pos_class_id')], axis=1)

In [342]:
train_columns = meta_trainval_2.columns.tolist()
train_columns.remove('y_true')
train_columns.remove('target')

Trainval / Test split

In [343]:
from sklearn.model_selection import StratifiedShuffleSplit

In [344]:
seed = 31500
split = StratifiedShuffleSplit(random_state=seed, test_size=0.3)

In [345]:
x_total = meta_trainval_2[train_columns].values
y_total = meta_trainval_2['target'].values

In [346]:
for train_index, test_index in split.split(x_total, y_total):
    break

In [347]:

x_trainval = x_total[train_index]
y_trainval = y_total[train_index]

x_test = x_total[test_index]
y_test = y_total[test_index]

GBM as meta-model

In [348]:
import lightgbm as lgb

In [349]:
lgb_trainval = lgb.Dataset(x_trainval, label=y_trainval)

In [367]:
params = {
    'boosting_type': 'gbdt',
    'objective': 'binary',
    'metric': 'binary_logloss',
    'num_leaves': 31,
    'max_depth': 15,
    'learning_rate': 0.005,
    'feature_fraction': 0.9,
    'bagging_fraction': 0.8,
    'bagging_freq': 5,
}

In [368]:
num_boost_round = 2500

In [369]:
cv_results = lgb.cv(params, lgb_trainval, num_boost_round=num_boost_round, nfold=5, 
                    metrics='binary_error',                    
                    early_stopping_rounds=100, verbose_eval=50)

[50]	cv_agg's binary_error: 0.152153 + 0.00606558
[100]	cv_agg's binary_error: 0.146816 + 0.00688565
[150]	cv_agg's binary_error: 0.13864 + 0.00705357
[200]	cv_agg's binary_error: 0.135916 + 0.00789785
[250]	cv_agg's binary_error: 0.133644 + 0.00631278
[300]	cv_agg's binary_error: 0.133531 + 0.0083558
[350]	cv_agg's binary_error: 0.131146 + 0.00943185
[400]	cv_agg's binary_error: 0.13001 + 0.00876638
[450]	cv_agg's binary_error: 0.128421 + 0.00840288
[500]	cv_agg's binary_error: 0.12774 + 0.00673413
[550]	cv_agg's binary_error: 0.126604 + 0.00747943
[600]	cv_agg's binary_error: 0.126263 + 0.00753783
[650]	cv_agg's binary_error: 0.127626 + 0.00759669
[700]	cv_agg's binary_error: 0.12808 + 0.00751153


In [370]:
best_boost_round = np.argmin(cv_results['binary_error-mean'])

In [371]:
gbm = lgb.train(params, lgb_trainval, num_boost_round=best_boost_round)

In [372]:
y_test_probas = gbm.predict(x_test)

In [373]:
y_test_pred = (y_test_probas > 0.45).astype(np.int)

In [374]:
y_test_pred.shape, y_test.shape

((3775,), (3775,))

In [375]:
from sklearn.metrics import accuracy_score

accuracy_score(y_test, y_test_pred)

0.8603973509933774

In [376]:
y_test_true = meta_trainval_['y_true'].values[test_index]

In [380]:
pos_class_id_test = x_test[:, 10].astype(np.int)
top1_pred_test = x_test[:, 4].astype(np.int)

In [381]:
mask = y_test_pred > 0

In [382]:
pos_class_id_test[mask], y_test_true[mask]

(array([ 79,  35, 125, ...,  76,  59,  12]),
 array([ 79,  35, 125, ...,  76,  59,  12]))

In [383]:
accuracy_score(y_test_true[mask], pos_class_id_test[mask])

0.8275862068965517

In [384]:
accuracy_score(y_test_true[mask], top1_pred_test[mask])

0.8991250643335049

In [103]:
best_params, trials = hp_optimize(hp_score, model_hp_params, max_evals=n_trials)
best_params.update(model_params)

print("Best parameters: \n{}".format(best_params))
print("Best trial : \n{}".format(trials.best_trial))

0:	learn: -4.7117042	total: 20.6s	remaining: 1m 22s
0:	learn: -4.7280812	total: 20.7s	remaining: 1m 22s
0:	learn: -4.7249822	total: 20.9s	remaining: 1m 23s
0:	learn: -4.6774325	total: 22.2s	remaining: 1m 28s
0:	learn: -4.7068981	total: 23.7s	remaining: 1m 34s
1:	learn: -4.5168791	total: 41.5s	remaining: 1m 2s
1:	learn: -4.5250657	total: 41.7s	remaining: 1m 2s
1:	learn: -4.6246349	total: 41.8s	remaining: 1m 2s
1:	learn: -4.6276355	total: 42.6s	remaining: 1m 3s
1:	learn: -4.5363106	total: 46.8s	remaining: 1m 10s
2:	learn: -4.4961720	total: 1m 2s	remaining: 41.9s
2:	learn: -4.3414154	total: 1m 3s	remaining: 42.2s
2:	learn: -4.4109650	total: 1m 3s	remaining: 42.3s
2:	learn: -4.4711836	total: 1m 5s	remaining: 43.8s
2:	learn: -4.4313898	total: 1m 7s	remaining: 44.9s
3:	learn: -4.3371345	total: 1m 24s	remaining: 21s
3:	learn: -4.3623223	total: 1m 25s	remaining: 21.3s
3:	learn: -4.2483936	total: 1m 26s	remaining: 21.6s
3:	learn: -4.3404164	total: 1m 27s	remaining: 21.8s
3:	learn: -4.2623334	to

Process ForkPoolWorker-477:
Process ForkPoolWorker-476:
Process ForkPoolWorker-478:
Traceback (most recent call last):
Process ForkPoolWorker-480:
Traceback (most recent call last):
Process ForkPoolWorker-479:
Traceback (most recent call last):
  File "/usr/lib/python3.5/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
Traceback (most recent call last):
Traceback (most recent call last):
  File "/usr/lib/python3.5/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
  File "/usr/lib/python3.5/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
  File "/usr/lib/python3.5/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
  File "/usr/lib/python3.5/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python3.5/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
  File "/usr/lib/python3.5/multiprocessing/process.py", line 93, in run
    self._target(*se

KeyboardInterrupt: 